## Final Exam Machine Learning

Davide Tateo; 167275

Francesca Salute; 167284

Nicole Favero; 167340 

Tomás Gonçalves; 167288

In [8]:
#importing required API and modules
import warnings
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import img_to_array
from keras.preprocessing.image import array_to_img
from tensorflow.keras.utils import load_img

In [14]:
def load_images_to_df(base_path):
    data = {'photo_id': [], 'image': [], 'image_array' :[], 'malignant': []}
    categories = {'Benign': 0, 'Malignant': 1}
    
    for subset in ['train', 'test']:
        for category in ['Benign', 'Malignant']:
            folder_path = os.path.join(base_path, subset, category)
            for filename in os.listdir(folder_path):
                if filename.endswith('.jpg'):
                    file_path = os.path.join(folder_path, filename)
                    # Upload Images
                    image = load_img(file_path)
                    image_array = img_to_array(image)
                    # Add the data
                    data['photo_id'].append(filename)
                    data['image'].append(image)
                    data['image_array'].append(image_array)
                    data['malignant'].append(categories[category])
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df


base_path = './images'
df = load_images_to_df(base_path)


In [13]:
print(df.head())
row_count = len(df)
print("Number of rows in the DataFrame:", row_count)

   photo_id                                              image  \
0     1.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
1    10.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
2   100.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
3  1000.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
4  1001.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   

                                         image_array  malignant  
0  [[[181.0, 140.0, 144.0], [190.0, 149.0, 153.0]...          0  
1  [[[231.0, 138.0, 156.0], [232.0, 139.0, 157.0]...          0  
2  [[[144.0, 126.0, 138.0], [151.0, 133.0, 145.0]...          0  
3  [[[238.0, 150.0, 174.0], [238.0, 150.0, 174.0]...          0  
4  [[[211.0, 122.0, 142.0], [213.0, 124.0, 144.0]...          0  
Number of rows in the DataFrame: 13879
